# Lab 3: Policy Search
Solution written by Serra Matteo and Matteo Magnaldi.

In [2]:
import logging
#from minmax import *
from functools import reduce
from math import inf
from copy import deepcopy
from functools import cache
import random

In [408]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self.safe = self.evaluate()
        self.winner = 0  
    
    
    def niming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        if self._k:
            #print(f"k:{self._k},num_obj:{num_objects}")
            assert num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.info("Yeuch!")

    def evaluate(self) -> bool:
        safe = reduce((lambda x,y: x ^ y), self._rows)
        self._safe = safe == 0
        return self._safe
    
    def is_finished(self) -> bool:
        return all(map(lambda x: x == 0, self._rows))
    
    def __hash__(self):
        return tuple(self._rows).__hash__()


def random_move(nim:Nim):
    #select a random valid row (!= 0) and select a random number of element to remove
    rand_row = random.randint(0,len(nim._rows)-1)
    while nim._rows[rand_row] == 0:
        rand_row = random.randint(0,len(nim._rows)-1)
    #if the nim is without k the limit is the value of the row
    k = nim._k if nim._k else nim._rows[rand_row]
    if k < nim._rows[rand_row]:
        rand_numObj = random.randint(1,k)
    else:
        rand_numObj = random.randint(1,nim._rows[rand_row])
    return rand_row,rand_numObj

def nonZeroRows(nim:Nim):
    #count the number of rows != 0
    return len(nim._rows) - [n == 0 for n in nim._rows].count(True)

def minSum(rows):
    return reduce((lambda x,y: x ^ y), rows)

def subVersion(nim:Nim): 
    #invert_strategy = [n > 1 for n in nim._rows].count(True) == 1 and max(nim._rows) <= nim._k
    invert_strategy = False    
    modRows = [row % (nim._k+1) for row in nim._rows ]
    safe= minSum(modRows) == 0
    if all([n == 1 or n == 0 for n in nim._rows]):
        return random_move(nim)
    else:
        if invert_strategy:
            #not used anymore because now who take the last win
            #when the number of rows whith one remain element will be odd, remove n elements to have all rows with one
            maxElem = max(nim._rows)  
            if nonZeroRows(nim) % 2 == 0 :
                return nim._rows.index(maxElem),maxElem-1
            else:
                return nim._rows.index(maxElem),maxElem
        else:
            if safe:
                #search for the best move to do, for witch the opponent can't bring us back to safe, if not found do a random move
                for i,n in enumerate(nim._rows):
                    for k in range(1,nim._k+1):
                        if n<k:
                            continue
                        temp_nim = nim._rows[:]
                        temp_nim[i] -= k
                        if minCost(temp_nim)[1] > nim._k:
                            #print("trovata")
                            return i,k
                return random_move(nim)
            else:
                #best move to do
                return minCost(modRows)

def expert_system(nim: Nim) -> (int, int):
    #expert system basted on nimsum
    if nim._k:
        return subVersion(nim)
    else:
        #if no k is used
        safe= minSum(nim._rows) == 0
        if all([n == 1 or n == 0 for n in nim._rows]):
            return random_move(nim)
        else:
            if safe:
                return random_move(nim)
            else:
                # comeback to safe
                return minCost(nim._rows)


def randomBanchmark(n_iter,n,p1_strategy,p2_strategy,k=None):
    # play a n_iter number of match and count the % of victory of the first strategy
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += play(nim,p1_strategy,p2_strategy)
    print(f"winner ratio: {v/n_iter*100}") 
    return 1 if v/n_iter*100 >= 50.00 else 0

def play(nim: Nim,p1_strategy,p2_strategy):
    #play a match and return the winner
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        if turn:
            row,k = p2_strategy(nim)
        else:
            row,k = p1_strategy(nim)
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)

def minCost(arr):
     
    cost = inf
    elem = -1
    # calculate XOR sum of array
    XOR = 0
    for e in arr:
         XOR ^= e
 
    # find the min cost and element
    # corresponding
    r_cost = 0
    for i,e in enumerate(arr):
        new_cost = abs((XOR ^ e) - e)
        if (cost > new_cost):
            r_cost = ((XOR ^ e) - e)
            if ((XOR ^ e) - e) > 0: 
                continue
            else:
                cost = (XOR ^ e) - e
            element = e
 
    #print("Element = ", element)
    #print("Operation required = ", r_cost)
    return arr.index(element),-r_cost



In [358]:
nim = Nim(6,3)
random_move(nim)

(4, 2)

In [362]:
#test on the expert system
randomBanchmark(10000,8,expert_system,random_move)
randomBanchmark(10000,8,random_move,random_move)
randomBanchmark(10000,8,expert_system,expert_system)


winner ratio: 100.0
winner ratio: 50.029999999999994
winner ratio: 50.14999999999999


1

In [435]:

NUM_GENERATIONS = 100
POPULATION_SIZE = 30
NIM_DIM = 6
K = 4
NUM_ITER = 200
NUM_ROUND = 200
NUM_COOK_STATUS = 8

def cook_status(nim:Nim):
    #set of possible move with the different strategies
    cooked = []
    nim_without_zero = [e for e in nim._rows if e !=0 ]
    k = nim._k if nim._k else max(nim._rows)
    #longest_row_max_take
    cooked.append((nim._rows.index(max(nim._rows)), k if k< max(nim._rows) else max(nim._rows)))
    #longest_row_min_take
    cooked.append((nim._rows.index(max(nim._rows)), 1))
    #shortest_row_min_take
    cooked.append((nim._rows.index(min(nim_without_zero)), 1))
    #leave one or remove max row
    if not nim._k :
        num_one_rows = nim._rows.count(1)
        cooked.append((nim._rows.index(max(nim._rows)), max(nim._rows) if num_one_rows %2 ==0 else max(nim._rows)-1))
    #longest_row_rand_take
    cooked.append((nim._rows.index(max(nim._rows)),random.randint(1,min(k,max(nim._rows)))))
    #shortest_row_rand_take
    k = nim._k if nim._k else min(nim_without_zero)
    cooked.append((nim._rows.index(min(nim_without_zero)),random.randint(1,min(k,min(nim_without_zero)))))
    #shortest_row_max_take
    cooked.append((nim._rows.index(min(nim_without_zero)), k if k< min(nim_without_zero) else min(nim_without_zero)))
    #random_move
    cooked.append(random_move(nim))
    #print(cooked)
    return cooked

def make_stategy(nim:Nim,probabilities):
    #select one strategy relying on the probabilities
    data = cook_status(nim)
    # if round(sum(probabilities),0)!= 1.0:
    #     print("Probabilità errate")
    #     return None
    random_number = random.random()
    prec_probability = 0.0
    for i,e in enumerate(probabilities):
        if random_number < prec_probability + e :
            #print(f"strategy selected {i}")
            return data[i]
        prec_probability += e
    return data[-1]

def playWithProb(nim: Nim,p1_prob,p2_prob):
    #play a match using the prob array
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        if turn:
            row, k = make_stategy(nim,p2_prob)
        else:
            row,k = make_stategy(nim,p1_prob)
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)

def randomBanchmarkWithProb(n_iter,n,p1_prob,p2_prob,k=None):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playWithProb(nim,p1_prob,p2_prob)
    #print(f"winner ratio: {v/n_iter*100}")
    return 1 if v/n_iter*100 >= 50.00 else 0

def generateRandomProbability():
    #generate a random set of ptobabilities for the strategies
    prob = []
    max_range = 1.0
    for i in range(NUM_COOK_STATUS):
        rand_num = random.uniform(0,max_range)
        max_range -= rand_num
        prob.append(rand_num)
    return prob

def tournament(population,tournamen_size = 2):
    #select the element with highest victory
    return population.index(max(random.choices(population,k=tournamen_size), key=lambda i: i[1]))

def evolutionWithProb():
    #start with a random population
    #select two stategies and see whitch is the best and increment the victory count of that stategy
    #do it for NUM_ROUND times
    #discard the worst
    #reset the victory counters 
    #add new element to the population
    population = [(generateRandomProbability(),0) for _ in range(POPULATION_SIZE)]
    for i in range(NUM_GENERATIONS):
        print(i)
        for _ in range(NUM_ROUND):
            index_p1 = tournament(population)
            index_p2 = tournament(population)
            #print(f"p1:{p1[0]},p2:{p2[0]}")
            if randomBanchmarkWithProb(NUM_ITER,NIM_DIM,population[index_p1][0],population[index_p2][0]):
                population[index_p1] = (population[index_p1][0],population[index_p1][1] + 1)
            else: 
                population[index_p2] = (population[index_p2][0],population[index_p2][1] + 1)
            # print(population[index_p1])
            # print(population[index_p1])
        #print(f"population:{population}")
        population = sorted(population,key=lambda i: i[1],reverse=True)[:POPULATION_SIZE//2]
        population = [(e[0],0) for _,e in enumerate(population)]
        population += [(generateRandomProbability(),0) for _ in range(POPULATION_SIZE//2)]
    print(f"best strategy {population[0]}")
    return population[0][0]


In [436]:
best_strategy = evolutionWithProb()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
best strategy ([0.4246964992214297, 0.536838986762267, 0.033724216399176096, 0.004546491921412203, 4.4140139586690196e-05, 0.00010530735405659253, 3.98005978338359e-05, 2.688872152348148e-06], 0)


In [437]:
#test on the best strategy found
def test(n_iter,n,p1_prob,p2_prob,k=None):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playWithProb(nim,p1_prob,p2_prob)
    print(f"winner ratio: {v/n_iter*100}")
    #return 1 if v/n_iter*100 >= 50.00 else 0
test(10000,6,best_strategy,[0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0])
test(10000,6,best_strategy,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy,[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy,[0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy,[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy,[0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0])
test(10000,6,best_strategy,[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0])
test(10000,6,best_strategy,[0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0])

winner ratio: 49.76
winner ratio: 64.49000000000001
winner ratio: 46.53
winner ratio: 72.48
winner ratio: 98.69
winner ratio: 58.84
winner ratio: 30.09
winner ratio: 50.68


In [422]:
Q = 0.01
NUM_ROUND = 1000
NIM_DIM = 6
K = 4
NUM_ITER = 1000

def select_stategies():
    #choose two random strategies for the list
    p1_strategy = random.randint(0,NUM_COOK_STATUS-1)
    p2_strategy = random.randint(0,NUM_COOK_STATUS-1)
    while p2_strategy == p1_strategy:
        p2_strategy = random.randint(0,NUM_COOK_STATUS-1)
    return p1_strategy,p2_strategy
    

def playWithStrategy(nim: Nim,p1_strategy,p2_strategy):
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        data = cook_status(nim)
        if turn:
            row, k = data[p2_strategy]
        else:
            row,k = data[p1_strategy]
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)

def randomBanchmarkWithStrategy(n_iter,n,p1_strategy,p2_strategy,k=None):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playWithStrategy(nim,p1_strategy,p2_strategy)
    #print(f"winner ratio: {v/n_iter*100}")
    return 1 if v/n_iter*100 >= 50.00 else 0
    

def evolutionWithStrategy():
    #all strategies start with the same probability
    #select two strategies and play
    #if win increment his probability and decrement the other
    probabilities = [1/NUM_COOK_STATUS for _ in range(NUM_COOK_STATUS)]
    #for _ in range(NUM_GENERATIONS):
    for _ in range(NUM_ROUND):
            p1_strategy,p2_strategy = select_stategies()
            if randomBanchmarkWithStrategy(NUM_ITER,NIM_DIM,p1_strategy,p2_strategy):
                if probabilities[p1_strategy] + Q < 1 and probabilities[p2_strategy] - Q >= 0:
                    probabilities[p1_strategy] += Q
                    probabilities[p2_strategy] -= Q
            else:
                if probabilities[p2_strategy] + Q < 1 and probabilities[p1_strategy] - Q >= 0:
                    probabilities[p2_strategy] += Q
                    probabilities[p1_strategy] -= Q
            # print(population[index_p1])
            # print(population[index_p1])
        #print(f"population:{population}")
    print(f"best strategy {probabilities}")
    return probabilities

In [423]:
best_strategy_2 = evolutionWithStrategy()

best strategy [0.045000000000000026, 0.005000000000000022, 0.005000000000000022, 0.005000000000000022, 0.015000000000000022, 0.005000000000000022, 0.9150000000000007, 0.005000000000000022]


In [424]:
#test on the best strategy found 
test(10000,6,best_strategy_2,[0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0])
test(10000,6,best_strategy_2,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy_2,[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy_2,[0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy_2,[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0])
test(10000,6,best_strategy_2,[0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0])
test(10000,6,best_strategy_2,[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0])
test(10000,6,best_strategy_2,[0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0])
test(10000,6,best_strategy,best_strategy_2)

winner ratio: 78.45
winner ratio: 50.32
winner ratio: 99.6
winner ratio: 99.75
winner ratio: 56.46
winner ratio: 89.8
winner ratio: 49.44
winner ratio: 85.00999999999999
winner ratio: 45.379999999999995


In [324]:
#test against the expert system
def playProbvsStrategy(nim: Nim,p1_strategy,p2_strategy):
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        if turn:
            row, k = p2_strategy(nim)
        else:
            row,k = make_stategy(nim,p1_strategy)
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)

def randomBanchmarkProbvsStrategy(n_iter,n,p1_prob,p2_prob,k=None):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playProbvsStrategy(nim,p1_prob,p2_prob)
    print(f"winner ratio: {v/n_iter*100}")
    #return 1 if v/n_iter*100 >= 50.00 else 0

randomBanchmarkProbvsStrategy(10000,8,6,best_strategy_2,expert_system)

winner ratio: 0.0


0

In [477]:
import numpy
nim = Nim(5)
#print(nim._rows)
G = {}
def init_state(nim_rows,states,row):
    if row>=len(nim_rows):
        return
    for k in range(0,nim_rows[row]+1):
        new_state = nim_rows[:row] + [nim_rows[row] - k] + nim_rows[row+1:]
        #print(new_state)
        states[tuple(new_state)] = numpy.random.uniform(low=1.0, high=0.1)
        init_state(new_state,states,row+1)
init_state(nim._rows,G,0)

In [541]:
class Agent(object):
    def __init__(self, states, alpha=0.15, random_factor=0.2):  # 80% explore, 20% exploit
        self.state_history = [(tuple(states),0)]  # state, reward
        self.alpha = alpha
        self.random_factor = random_factor
        self.G = {}
        self.init_reward(states,0)

    def init_reward(self,nim_rows,row):
        if row>=len(nim_rows):
            return
        for k in range(0,nim_rows[row]+1):
            new_state = nim_rows[:row] + [nim_rows[row] - k] + nim_rows[row+1:]
            #print(new_state)
            self.G[tuple(new_state)] = numpy.random.uniform(low=1.0, high=0.1)
            self.init_reward(new_state,row+1)

    def get_reward(self,state):
        return self.G[state]

    def choose_action(self, state, allowedMoves):
        maxG = -10e15
        next_move = None
        randomN = numpy.random.random()
        if randomN < self.random_factor:
            # if random number below random factor, choose random action
            next_move = random_move(state)
        else:
            # if exploiting, gather all possible actions and choose one with the highest G (reward)
            for (row,k) in allowedMoves:
                #new_state = tuple([sum(x) for x in zip(state, ACTIONS[action])])
                new_state = state._rows[:row] + [state._rows[row] - k] + state._rows[row+1:]
                if self.G[tuple(new_state)] >= maxG:
                    next_move = (row,k)
                    maxG = self.G[tuple(new_state)]

        return next_move

    def update_state_history(self, state, reward):
        #print(f"update state history: state: {state} reward: {reward}")
        self.state_history.append((tuple(state), reward))

    def learn(self):
        target = 0
        #print(f"learn: {self.state_history}")
        for prev, reward in reversed(self.state_history):
            #print(prev,reward)
            prev = tuple(prev)
            self.G[prev] = self.G[prev] + self.alpha * (target - self.G[prev])
            target += reward

        self.state_history = []
        
        self.random_factor -= 10e-5  # decrease random factor each episode of play


In [574]:
NIM_DIM = 5
NUM_ROUND = 10000

def opponent_move(nim:Nim):
    row,k = random_move(nim)
    nim.niming(row,k)
    #print(f"opponent move: row:{row} k:{k}")

def possible_actions(nim:Nim):
    actions = []
    for i,row in enumerate(nim._rows):
        for k in range(1,row+1):
            actions.append((i,k))
    return actions

def playWithRobot(nim:Nim,robot,opponent_strategy):
    turn = random.randint(0,1)
    #nim = Nim(NIM_DIM)
    while not nim.is_finished():
        if turn:
            row,k = robot.choose_action(nim,possible_actions(nim))
            nim.niming(row,k)
        else:
            opponent_strategy(nim)
        turn = not turn
    return 1 if not turn else 0

def randomBanchmarkWithRobot(n_iter,n,robot,opponent_strategy):
    v = 0
    for i in range(n_iter):
        nim = Nim(n)
        v += playWithRobot(nim,robot,opponent_strategy)
    print(f"winner ratio: {v/n_iter*100}")

def evolution():
    nim = Nim(NIM_DIM)
    robot = Agent(nim._rows,alpha=0.1,random_factor=0.4)

    for i in range(NUM_ROUND):
        turn = random.randint(0,1)
        nim = Nim(NIM_DIM)
        #print(nim._rows)
        #print(turn)
        while not nim.is_finished():
            if turn:
                row,k = robot.choose_action(nim,possible_actions(nim))
                #print(f"robot move: row:{row} k:{k}")
                nim.niming(row,k)
                reward = 1 if nim.is_finished() else 0
                robot.update_state_history(nim._rows, reward)
            else:
                opponent_move(nim)
                reward = -1 if nim.is_finished() else 0
                robot.update_state_history(nim._rows,reward)
            turn = not turn
        robot.learn()  # robot should learn after every episode
            # get a history of number of steps taken to plot later
        if i % 1000 == 0:
            randomBanchmarkWithRobot(10000,NIM_DIM,robot,opponent_move)
    return robot


In [577]:
for i in range(10):
    print(f"Run number: {i}")
    robot = evolution()

run number: 0
winner ratio: 39.92
winner ratio: 45.39
winner ratio: 57.03
winner ratio: 62.17
winner ratio: 66.81
winner ratio: 50.949999999999996
winner ratio: 54.39000000000001
winner ratio: 67.65
winner ratio: 65.74
winner ratio: 43.02
winner ratio: 41.05
winner ratio: 64.67
winner ratio: 48.11
winner ratio: 55.900000000000006
winner ratio: 53.22
winner ratio: 52.66
winner ratio: 55.300000000000004
winner ratio: 60.73
winner ratio: 56.45
winner ratio: 38.33
run number: 1
winner ratio: 47.8
winner ratio: 48.980000000000004
winner ratio: 46.17
winner ratio: 50.44
winner ratio: 49.95
winner ratio: 44.83
winner ratio: 50.82
winner ratio: 58.91
winner ratio: 46.050000000000004
winner ratio: 65.41
winner ratio: 58.330000000000005
winner ratio: 58.77
winner ratio: 68.61
winner ratio: 52.14
winner ratio: 60.440000000000005
winner ratio: 50.629999999999995
winner ratio: 48.33
winner ratio: 64.7
winner ratio: 50.59
winner ratio: 62.27
run number: 2
winner ratio: 57.92
winner ratio: 57.9
winne